In [1]:
import pandas as pd

df_checks = pd.read_csv('./lentahack/20210518_checks.csv')
df_uplift = pd.read_csv('./lentahack/20210518_uplift.csv')
df_sample_submission = pd.read_csv('./lentahack/20210521_sample_submission.csv')
df_hierarchy = pd.read_csv('./lentahack/20210518_hierarchy.csv')
df_offers = pd.read_csv('./lentahack/20210521_offers.csv')

In [2]:
from datetime import datetime, timedelta
import numpy as np

In [3]:
df_checks.head()

,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,region_name
0,d9da50b77962ad6401d0527db5b65b04,20191230,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b17c870027bb4a22e3aedb971bc00def,b3deb0286313f0b888c0eac49580cc23,1,5.0,133.40,249.45,152f1b77a32508570e2745daf9ce7aec
1,d9da50b77962ad6401d0527db5b65b04,20191230,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b9ef00e971d0fb459a765708efba70c0,680a96609351230ed00e1d1090371cbc,2,1.0,96.77,118.99,152f1b77a32508570e2745daf9ce7aec
2,d9da50b77962ad6401d0527db5b65b04,20191230,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b75147feae4cf8ec07fb5f2cef855def,7215ee9c7d9dc229d2921a40e899ec5f,5,2.0,2.34,7.58,152f1b77a32508570e2745daf9ce7aec
3,d9da50b77962ad6401d0527db5b65b04,20191230,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b25e0d42165d246d23cbf4d3efcdf583,7215ee9c7d9dc229d2921a40e899ec5f,4,1.0,23.59,39.99,152f1b77a32508570e2745daf9ce7aec
4,d9da50b77962ad6401d0527db5b65b04,20191230,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,a537d2eef30d47f9b11d655421ca93fb,05c93d4fa9bdbe70b5d2204ce9548ca9,3,1.0,24.02,39.99,152f1b77a32508570e2745daf9ce7aec


In [4]:
df_uplift.head()

,Offer_ID,UpLift
0,241,1.743427
1,70,16.870948
2,342,15.185624
3,B33,0.921722
4,244,4.862729


In [5]:
df_hierarchy.head()

,sku,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,29285fb39582e5a63c14026198c6ad51,44a7b825383402ed937a15fb8a5bb12a,9a1158154dfa42caddbd0694a4e9bdc8,f50c7035e532c49a0f6993d988e2e843,e9f41ed96e63ca1be6aea099625afe6f
1,f31d0af494b191ef2bc8395db9c8366f,44a7b825383402ed937a15fb8a5bb12a,ad61ab143223efbc24c7d2583be69251,84e8ce7870f0eecd843366582bb95a28,b7fa186535a75c869bfaece622c72689
2,49edad6669ba09dffbf278b76c53cd9d,cc9284597026dcc021735c0970ca0c93,1c383cd30b7c298ab50293adfecb7b18,2a3d6d6cc4b5e77238c1fc1bb6cdd681,7e0e0c1fe174b2d2efbf5ae0e2bc3ee9
3,c5c632eb0f93a233ef6db2620a464ea0,44a7b825383402ed937a15fb8a5bb12a,d82c8d1619ad8176d665453cfb2e55f0,b27d5296bede63b1493a5d321d4e8092,ec2acf8ebfdb679d5fc87caadd98fb58
4,9d0c7880f16c7748e1e992b28655c67f,cc9284597026dcc021735c0970ca0c93,6ea9ab1baa0efb9e19094440c317e21b,379a7ba015d8bf1c70b8add2c287c6fa,5aec2478fac4d1ee912c794c343d036c


In [6]:
df_offers.sort_values('Offer_ID')

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group
3687,Biweekly,10,c4b0a5d6be61feef47a33f8230beec38,20200402,20200415,train
3214,Biweekly,10,66dd48b798614b7d193a83777c8a0fdc,20200402,20200415,train
3609,Biweekly,10,f5cac4fb776999712e9413fb7890ba4b,20200402,20200415,train
3854,Biweekly,10,f049d165a494530cceac3252483da913,20200402,20200415,train
3964,Biweekly,10,953fce3d637849721d74508251737be2,20200402,20200415,train
...,...,...,...,...,...,...
2471,Facades,F98,7a8675a8f6988ebcbefd062fabaccc85,20200618,20200624,train
2427,Facades,F98,ef7bf412117548cb80b3f3b33a12977d,20200618,20200624,train
3059,Facades,F98,5abc0869d756fa1c6831651776138baa,20200618,20200624,train
2935,Facades,F99,5e3039a269ff005f2c76cea16c0cf783,20200618,20200624,train


In [7]:
df_offers_upd = df_offers
df_offers_upd['start_date'] = pd.to_datetime(df_offers_upd['start_date'], format='%Y%m%d')
df_offers_upd['end_date'] = pd.to_datetime(df_offers_upd['end_date'], format='%Y%m%d')
df_offers_upd

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group
0,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train
1,Facades,F2,e87788b585ead792a4d40f7de18f2f76,2020-01-02,2020-01-08,train
2,Facades,F9,5ae92b5e465d630285677e80b43fe5f7,2020-01-16,2020-01-22,train
3,Facades,F45,0790b472112a765339f0e5c3f66816ee,2020-03-19,2020-03-25,train
4,Facades,F191,e87788b585ead792a4d40f7de18f2f76,2020-11-12,2020-11-18,test
...,...,...,...,...,...,...
9771,Biweekly,683,0f368be7897dd1531e43dc37115eeaf0,2019-11-14,2019-11-27,train
9772,Biweekly,691,47c28bf957fe120f2d016e46911e5bdb,2019-11-14,2019-11-27,train
9773,Biweekly,683,00432d021eca6059fee2ed69a732c2b2,2019-11-14,2019-11-27,train
9774,Biweekly,690,40d78b3648a4ea15f300070a0ffd9752,2019-11-14,2019-11-27,train


In [8]:
df_checks_upd = df_checks.drop(['client_id','shop_id','time','check_pos','region_name'], axis=1)
df_checks_upd['day'] = pd.to_datetime(df_checks_upd['day'], format='%Y%m%d')
df_checks_upd

,day,check_id,sku,promo_id,num_sales,supplier_price,selling_price
0,2019-12-30,c0b3f9c2215f924e27a6039617999c60,b17c870027bb4a22e3aedb971bc00def,b3deb0286313f0b888c0eac49580cc23,5.0,133.40,249.45
1,2019-12-30,c0b3f9c2215f924e27a6039617999c60,b9ef00e971d0fb459a765708efba70c0,680a96609351230ed00e1d1090371cbc,1.0,96.77,118.99
2,2019-12-30,c0b3f9c2215f924e27a6039617999c60,b75147feae4cf8ec07fb5f2cef855def,7215ee9c7d9dc229d2921a40e899ec5f,2.0,2.34,7.58
3,2019-12-30,c0b3f9c2215f924e27a6039617999c60,b25e0d42165d246d23cbf4d3efcdf583,7215ee9c7d9dc229d2921a40e899ec5f,1.0,23.59,39.99
4,2019-12-30,c0b3f9c2215f924e27a6039617999c60,a537d2eef30d47f9b11d655421ca93fb,05c93d4fa9bdbe70b5d2204ce9548ca9,1.0,24.02,39.99
...,...,...,...,...,...,...,...
21479254,2020-01-01,ac8cc7468ed294fcecebdcfa00d4280f,dc155ba1e8a3c759bcf2f358384ba9cd,f1b40313f0dc51fde3fe2cc7bc2b987f,1.0,63.63,99.89
21479255,2020-01-01,ac8cc7468ed294fcecebdcfa00d4280f,305d39a0a0af37b5436e8a3ff3e9d319,357f92d49be136644805a94cdbeb462a,1.0,185.70,298.99
21479256,2020-01-01,ac8cc7468ed294fcecebdcfa00d4280f,b75147feae4cf8ec07fb5f2cef855def,7215ee9c7d9dc229d2921a40e899ec5f,1.0,1.14,3.49
21479257,2020-01-01,ac8cc7468ed294fcecebdcfa00d4280f,6540ad4c3e0b21d5a804b692dda6fe66,7215ee9c7d9dc229d2921a40e899ec5f,1.0,37.63,74.99


In [9]:
df_offers_checks = pd.merge(df_offers_upd, df_checks_upd, how='left', on=['sku'])
df_offers_checks

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,day,check_id,promo_id,num_sales,supplier_price,selling_price
0,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2019-12-30,6cd3a37302cf2874b590c05b881195ad,19695c56fe623b1e575fc340ee02f1d8,3.0,67.92,101.67
1,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2019-12-30,234522126f416909a33e6538330a2d03,19695c56fe623b1e575fc340ee02f1d8,1.0,20.51,33.89
2,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2019-12-29,f599fb5aabb014604667cb861ea58900,c17f9361b3a03dec01812d12cf9887ff,1.0,22.33,33.05
3,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2019-12-29,9953f543738c0a9695955e3edac3b546,19695c56fe623b1e575fc340ee02f1d8,2.0,41.54,67.78
4,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2019-12-29,162d473398b9fbecfd43e4257f8f807a,19695c56fe623b1e575fc340ee02f1d8,1.0,20.73,33.89
...,...,...,...,...,...,...,...,...,...,...,...,...
17821451,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,2019-11-14,2019-11-27,train,2019-10-02,1bdf82b4fa3aa57d2f8621cf46623263,7215ee9c7d9dc229d2921a40e899ec5f,1.0,41.30,96.69
17821452,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,2019-11-14,2019-11-27,train,2020-01-09,b0a664c8b09957225c41c73df58c43b3,7215ee9c7d9dc229d2921a40e899ec5f,2.0,81.90,193.38
17821453,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,2019-11-14,2019-11-27,train,2019-10-14,01b1f2cd54d059de5e01742c7837d55b,7215ee9c7d9dc229d2921a40e899ec5f,1.0,40.45,96.15
17821454,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,2019-11-14,2019-11-27,train,2019-10-21,d9a6aee2df46220c5616dea00fb43da2,406555edca620c40deaf451c61f4ebcf,1.0,40.40,39.99


In [10]:
df_oc = pd.merge(df_offers_upd, df_checks_upd, how='outer', on=['sku'])
df_oc

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,day,check_id,promo_id,num_sales,supplier_price,selling_price
0,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2019-12-30,6cd3a37302cf2874b590c05b881195ad,19695c56fe623b1e575fc340ee02f1d8,3.0,67.92,101.67
1,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2019-12-30,234522126f416909a33e6538330a2d03,19695c56fe623b1e575fc340ee02f1d8,1.0,20.51,33.89
2,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2019-12-29,f599fb5aabb014604667cb861ea58900,c17f9361b3a03dec01812d12cf9887ff,1.0,22.33,33.05
3,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2019-12-29,9953f543738c0a9695955e3edac3b546,19695c56fe623b1e575fc340ee02f1d8,2.0,41.54,67.78
4,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2019-12-29,162d473398b9fbecfd43e4257f8f807a,19695c56fe623b1e575fc340ee02f1d8,1.0,20.73,33.89
...,...,...,...,...,...,...,...,...,...,...,...,...
34337900,NaN,NaN,b010a834baf3fde030bd8880932c27bd,NaT,NaT,NaN,2020-01-01,b160945351d780571197af04d50d3aaf,7215ee9c7d9dc229d2921a40e899ec5f,1.0,125.00,199.99
34337901,NaN,NaN,9561d40cf8efbc71ca4f421a594c7f10,NaT,NaT,NaN,2020-01-01,fde19f3d9ac2bfa1a5637105159f3573,ebc3c1764f014e2e231782a3277e15b7,1.0,54.00,99.00
34337902,NaN,NaN,6c2a3fb4b7297e871c2a93b46965ae65,NaT,NaT,NaN,2020-01-01,6783a46b1c20f413852bfdf42b16050f,9676c2518834bb00606f309d37d40f9b,1.0,310.91,499.01
34337903,NaN,NaN,1712d86a6dfb02fc64359ba4c1554900,NaT,NaT,NaN,2020-01-01,c23adbbf3fff0ed6b907968ac8a690ec,922f1d28a2c4c6c0d07c2cb53271c860,1.0,221.84,299.06


In [11]:
# pre-period
duration = df_oc[df_oc['Offer_ID']=='241']['end_date'].values[0] - df_oc[df_oc['Offer_ID']=='241']['start_date'].values[0]
duration = int(duration.astype('timedelta64[D]')/np.timedelta64(1, 'D'))
pre = df_oc[(df_oc['day']>=df_oc['start_date']-timedelta(days=duration)) &
            (df_oc['day']<df_oc['start_date'])]
pre['profit'] = pre['selling_price']*pre['num_sales']
pre

<ipython-input-11-2f142ad2e2c3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre['profit'] = pre['selling_price']*pre['num_sales']


,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,day,check_id,promo_id,num_sales,supplier_price,selling_price,profit
52,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-08,af61335f9b4e7d84d341e67266ef0887,4e43ae37ca1284df3cafb0109129c114,1.0,21.78,29.99,29.99
54,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-08,7537ba3e33a46368fa78d61a6e1d1da6,4e43ae37ca1284df3cafb0109129c114,2.0,43.52,59.98,119.96
55,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-08,063b032dd95be9dae8e9cd7d62892283,4e43ae37ca1284df3cafb0109129c114,1.0,21.73,29.99,29.99
56,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-01,dd3b15724eb7486b35736cf7e16ce3bc,7215ee9c7d9dc229d2921a40e899ec5f,2.0,43.57,104.58,209.16
105,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-08,1507fb76c2c33e3de5c600cf3f3a19e5,4e43ae37ca1284df3cafb0109129c114,1.0,21.08,29.99,29.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17821350,Biweekly,690,dcb69d7f01aef0327bd4887d4629f74f,2019-11-14,2019-11-27,train,2019-11-03,50038e4ac1206b7dfbe0143fa78e8809,7215ee9c7d9dc229d2921a40e899ec5f,1.0,844.61,948.88,948.88
17821376,Biweekly,690,dcb69d7f01aef0327bd4887d4629f74f,2019-11-14,2019-11-27,train,2019-11-04,05c39262ee3d8435f8c9cbb3be944b26,7215ee9c7d9dc229d2921a40e899ec5f,2.0,1688.73,1993.16,3986.32
17821433,Biweekly,690,dcb69d7f01aef0327bd4887d4629f74f,2019-11-14,2019-11-27,train,2019-11-13,c7fd878fc6242a7d3a4d49a06916faec,7215ee9c7d9dc229d2921a40e899ec5f,1.0,844.31,1026.70,1026.70
17821434,Biweekly,690,dcb69d7f01aef0327bd4887d4629f74f,2019-11-14,2019-11-27,train,2019-11-05,4148eb86b97303ab16ba9736c84a6011,7215ee9c7d9dc229d2921a40e899ec5f,1.0,844.61,948.39,948.39


In [82]:
# promo period
during = df_oc[(df_oc['day']>=df_oc['start_date']) &
            (df_oc['day']<=df_oc['end_date'])]
during['profit'] = during['selling_price']*during['num_sales']
during

<ipython-input-82-b058372c8ace>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  during['profit'] = during['selling_price']*during['num_sales']


,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,day,check_id,promo_id,num_sales,supplier_price,selling_price,profit
37,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,ea9ef46e49abd4aeb22a1a4854ef4db8,dfe766bb394e01da9a64f6bfa74c978e,1.0,21.49,29.99,29.99
38,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,bca2ec0f3522f6dd73661e7e30ac53b4,dfe766bb394e01da9a64f6bfa74c978e,1.0,21.96,29.01,29.01
44,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,cd05aa5d30b7597bc3f7d8a283f41dbd,dfe766bb394e01da9a64f6bfa74c978e,1.0,21.75,29.99,29.99
45,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,7eaa330eabe7dd554c24940df08de667,dfe766bb394e01da9a64f6bfa74c978e,2.0,42.57,59.98,119.96
46,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,9642d16f8129fc3efa4a2a7eba98a1bd,dfe766bb394e01da9a64f6bfa74c978e,1.0,21.75,29.99,29.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17821422,Biweekly,690,dcb69d7f01aef0327bd4887d4629f74f,2019-11-14,2019-11-27,train,2019-11-25,0fd8d38a05fd887993dca67cbffc890f,b97aa2f46dc2372c52c276b4f4dc87a4,1.0,857.34,669.81,669.81
17821430,Biweekly,690,dcb69d7f01aef0327bd4887d4629f74f,2019-11-14,2019-11-27,train,2019-11-21,fab252383a87176342f0e80f7774e2cc,b97aa2f46dc2372c52c276b4f4dc87a4,1.0,870.93,669.81,669.81
17821447,Biweekly,690,dcb69d7f01aef0327bd4887d4629f74f,2019-11-14,2019-11-27,train,2019-11-26,5f6d1e2a80325b38801638400dd6a491,b97aa2f46dc2372c52c276b4f4dc87a4,1.0,886.43,669.42,669.42
17821448,Biweekly,690,dcb69d7f01aef0327bd4887d4629f74f,2019-11-14,2019-11-27,train,2019-11-25,3bd7b6c8c02e5bc48447096db2fce8be,b97aa2f46dc2372c52c276b4f4dc87a4,1.0,844.61,669.00,669.00


In [84]:
sum(during['profit'])/sum(pre['profit']) - 1

2.0218287185460064

In [18]:
# promo period
during = df_offers_checks[(df_offers_checks['day']>=df_offers_checks['start_date']) &
                (df_offers_checks['day']<=df_offers_checks['end_date'])]
offer_sku = pd.merge(during, df_uplift, how='left', on=['Offer_ID'])
offer_sku

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,day,check_id,promo_id,num_sales,supplier_price,selling_price,UpLift
0,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,ea9ef46e49abd4aeb22a1a4854ef4db8,dfe766bb394e01da9a64f6bfa74c978e,1.0,21.49,29.99,0.430811
1,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,bca2ec0f3522f6dd73661e7e30ac53b4,dfe766bb394e01da9a64f6bfa74c978e,1.0,21.96,29.01,0.430811
2,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,cd05aa5d30b7597bc3f7d8a283f41dbd,dfe766bb394e01da9a64f6bfa74c978e,1.0,21.75,29.99,0.430811
3,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,7eaa330eabe7dd554c24940df08de667,dfe766bb394e01da9a64f6bfa74c978e,2.0,42.57,59.98,0.430811
4,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,9642d16f8129fc3efa4a2a7eba98a1bd,dfe766bb394e01da9a64f6bfa74c978e,1.0,21.75,29.99,0.430811
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148581,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,2019-11-14,2019-11-27,train,2019-11-18,a0a1335d5a06f726f25b88526f29e6c6,62ee8f6806e64bb1e08c2bfc73cce473,1.0,41.00,68.89,5.097561
1148582,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,2019-11-14,2019-11-27,train,2019-11-18,c88e4f544e606ecb6647181ff07768d6,62ee8f6806e64bb1e08c2bfc73cce473,1.0,40.95,68.23,5.097561
1148583,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,2019-11-14,2019-11-27,train,2019-11-18,279d08cddabb95c98572048656d6f307,62ee8f6806e64bb1e08c2bfc73cce473,1.0,41.30,68.22,5.097561
1148584,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,2019-11-14,2019-11-27,train,2019-11-18,eb8b9be8ffdcd530a708f052f83e3c11,62ee8f6806e64bb1e08c2bfc73cce473,1.0,41.30,68.89,5.097561


In [19]:
offer_sku.to_csv('offer_sku.csv',index=False)

In [20]:
hierarchy = pd.merge(offer_sku, df_hierarchy, how='left', on=['sku'])
hierarchy

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,day,check_id,promo_id,num_sales,supplier_price,selling_price,UpLift,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,ea9ef46e49abd4aeb22a1a4854ef4db8,dfe766bb394e01da9a64f6bfa74c978e,1.0,21.49,29.99,0.430811,44a7b825383402ed937a15fb8a5bb12a,a5771bce93e200c36f7cd9dfd0e5deaa,37968ad196a5085f5ed91e58df8c2cef,4bc9b0056ab9c25ecd8794e2cdac48ad
1,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,bca2ec0f3522f6dd73661e7e30ac53b4,dfe766bb394e01da9a64f6bfa74c978e,1.0,21.96,29.01,0.430811,44a7b825383402ed937a15fb8a5bb12a,a5771bce93e200c36f7cd9dfd0e5deaa,37968ad196a5085f5ed91e58df8c2cef,4bc9b0056ab9c25ecd8794e2cdac48ad
2,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,cd05aa5d30b7597bc3f7d8a283f41dbd,dfe766bb394e01da9a64f6bfa74c978e,1.0,21.75,29.99,0.430811,44a7b825383402ed937a15fb8a5bb12a,a5771bce93e200c36f7cd9dfd0e5deaa,37968ad196a5085f5ed91e58df8c2cef,4bc9b0056ab9c25ecd8794e2cdac48ad
3,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,7eaa330eabe7dd554c24940df08de667,dfe766bb394e01da9a64f6bfa74c978e,2.0,42.57,59.98,0.430811,44a7b825383402ed937a15fb8a5bb12a,a5771bce93e200c36f7cd9dfd0e5deaa,37968ad196a5085f5ed91e58df8c2cef,4bc9b0056ab9c25ecd8794e2cdac48ad
4,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,2020-02-15,9642d16f8129fc3efa4a2a7eba98a1bd,dfe766bb394e01da9a64f6bfa74c978e,1.0,21.75,29.99,0.430811,44a7b825383402ed937a15fb8a5bb12a,a5771bce93e200c36f7cd9dfd0e5deaa,37968ad196a5085f5ed91e58df8c2cef,4bc9b0056ab9c25ecd8794e2cdac48ad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148581,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,2019-11-14,2019-11-27,train,2019-11-18,a0a1335d5a06f726f25b88526f29e6c6,62ee8f6806e64bb1e08c2bfc73cce473,1.0,41.00,68.89,5.097561,cc9284597026dcc021735c0970ca0c93,c16a5320fa475530d9583c34fd356ef5,62f91ce9b820a491ee78c108636db089,508648865001185b6a704e553c44b0d7
1148582,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,2019-11-14,2019-11-27,train,2019-11-18,c88e4f544e606ecb6647181ff07768d6,62ee8f6806e64bb1e08c2bfc73cce473,1.0,40.95,68.23,5.097561,cc9284597026dcc021735c0970ca0c93,c16a5320fa475530d9583c34fd356ef5,62f91ce9b820a491ee78c108636db089,508648865001185b6a704e553c44b0d7
1148583,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,2019-11-14,2019-11-27,train,2019-11-18,279d08cddabb95c98572048656d6f307,62ee8f6806e64bb1e08c2bfc73cce473,1.0,41.30,68.22,5.097561,cc9284597026dcc021735c0970ca0c93,c16a5320fa475530d9583c34fd356ef5,62f91ce9b820a491ee78c108636db089,508648865001185b6a704e553c44b0d7
1148584,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,2019-11-14,2019-11-27,train,2019-11-18,eb8b9be8ffdcd530a708f052f83e3c11,62ee8f6806e64bb1e08c2bfc73cce473,1.0,41.30,68.89,5.097561,cc9284597026dcc021735c0970ca0c93,c16a5320fa475530d9583c34fd356ef5,62f91ce9b820a491ee78c108636db089,508648865001185b6a704e553c44b0d7


In [21]:
hierarchy.to_csv('hierarchy.csv',index=False)

In [51]:
# pre-period
duration = df_offers_checks['end_date'].values[0] - df_offers_checks['start_date'].values[0]
duration = int(duration.astype('timedelta64[D]')/np.timedelta64(1, 'D'))
pre = df_offers_checks[(df_offers_checks['Offer_ID']=='241') & 
#                  (df_offers_checks['sku']=='71c770923102af052f64c6036c9dd590') & 
                 (df_offers_checks['day']>=df_offers_checks['start_date']-timedelta(days=duration)) &
                (df_offers_checks['day']<df_offers_checks['start_date'])]
pre['profit'] = pre['selling_price']*pre['num_sales']
pre

<ipython-input-51-7ada3fabe58d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre['profit'] = pre['selling_price']*pre['num_sales']


,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,day,check_id,promo_id,num_sales,supplier_price,selling_price,profit
5196838,Biweekly,241,ec155b6d970114030333664ad0a9d966,2020-04-16,2020-04-29,train,2020-04-11,8fdb0439f5ee8ec853ccbd863eb52da7,9d42978a93f7e3d1d6bfd33b63b79688,1.0,1348.5,1998.02,1998.02
5902054,Biweekly,241,67b703cefd7a948e0d58879ca1d0bf9b,2020-04-16,2020-04-29,train,2020-04-11,ac9b86034c032e30083ef246638ed052,9d42978a93f7e3d1d6bfd33b63b79688,2.0,2555.0,3597.98,7195.96
5902119,Biweekly,241,67b703cefd7a948e0d58879ca1d0bf9b,2020-04-16,2020-04-29,train,2020-04-12,81a786a69f64d110454f492436fcbd2f,9d42978a93f7e3d1d6bfd33b63b79688,1.0,1277.5,1798.92,1798.92
6541730,Biweekly,241,55d15801290453db237f3107a1534b75,2020-04-16,2020-04-29,train,2020-04-12,81cc2ca073d11a5e8610a9a8533c54bc,9d42978a93f7e3d1d6bfd33b63b79688,1.0,1419.5,1998.08,1998.08


In [52]:
sum(during['profit'])/sum(pre['profit']) - 1

3.93806163969154